# Lab 4: Text Feature Engineering on Azure


## Load the Gold Layer

In [0]:
# Configure access to your Azure Data Lake
spark.conf.set(
    "fs.azure.account.key.goodreadsreviewsgen2.dfs.core.windows.net",
    "WnTNNhXumIt06S3pwQEB8Vzkqh9D/VRtLq7ULy2OFppgW0zCb4QVwU1tthtk2Pz4HTxUNmHCXgC9+AStYAOSmQ=="
)

In [0]:
# Load the curated Gold dataset (features_v1)
gold_train_path = "abfss://lakehouse@goodreadsreviewsgen2.dfs.core.windows.net/gold/features_v1"

df = spark.read.format("delta").load(gold_train_path)

# Display schema and a few rows
df.printSchema()
display(df.limit(5))

root
 |-- book_id: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- author_id: string (nullable = true)
 |-- author_name: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- rating: float (nullable = true)
 |-- review_text: string (nullable = true)
 |-- language_code: string (nullable = true)
 |-- n_votes: integer (nullable = true)
 |-- date_updated: date (nullable = true)
 |-- review_length: integer (nullable = true)
 |-- review_length_in_words: integer (nullable = true)
 |-- average_rating: double (nullable = true)
 |-- number_of_reviews: long (nullable = true)



book_id,review_id,title,author_id,author_name,user_id,rating,review_text,language_code,n_votes,date_updated,review_length,review_length_in_words,average_rating,number_of_reviews
22726728,8af9a5405d8f7de5f787a0f60610782b,"adjournment (fate, #1)",8261567,emersyn vallis,65d3b49e7e4d1248f487b081c2c535c4,4.0,"sydney chandler choose to leave home, to give up her family and most of all to leave the love of her life simon behind. he broke her heart, he made his choice now she had to make hers. she chose herself. she left it all and started her life anew. she never really left simon though he was always in her heart. the love she felt for him from their childhood never diminished. when fate throws them back into each other's sights can sydney forgive and forget? can simon prove to her he's worth the risk? my review - 4 stars this story is about sidney chandler and simon mcallister and the love they built when they were very young growing up together. somehow along the way tragedy had simon making a tough decision that tore at him for years. the plot takes us through the story of how fate gives two young lovers a second chance to get it right. ten years after simon broke sidney's heart fate brings then together and simon will stop at nothing to win sidney's trust and her heart. i loved how simon did everything he could to get sidney to notice him, to get her to see how he never forgot their connection. he knows her better than she knows herself. he has never truly let her out of his sight, watching her from a far keeping her safe and waiting for the day they could finally be together. i absolutely loved the character of simon and the fact that he was a triplet made him all the more appealing. three hot alpha males to love and adore. i very much enjoyed the banter between the three chandler sisters and the three mcallister brothers. lots of sexual tension, a little less heat then i am used to in the books i read but when the sex scenes come they are well written and steamy. something happened to simon and sidney that the author hints at it seems everyone else remembers the accident but sidney has blocked it out a bit. that part of the story confused me a little there were some holes in the story but there are four more books so i am sure the author will piece it all together. this one ends in a bit of a cliffhanger. all the books in the series are out so no waiting you can get the whole story all at once. received an arc in exchange for an honest review.",eng,180,2016-06-14,2191,418,3.8048780487804876,41
8580703,c833fa735a07ae4513e9b438e99a10c0,in the dark streets shineth: a 1941 christmas eve story,6281688,david mccullough,bd9eac0385fa714845f5b883d7321db8,3.0,"#52!!! first time actually achieving my goal! under normal circumstances i might consider this tiny book cheating, but i'm nearly done with a really dumb book and didn't want it to be my 52nd. so i read this in the mean time.",,1214,2016-01-14,225,42,3.627906976744186,43
16704759,6d478aae9ad8545266ff706f35331739,"knight & stay (knight, #2)",6589173,kitty french,559d834709733a06774261fad3f67ac9,5.0,mmmmm mmmm mmmm,eng,18277,2013-07-21,15,3,4.455399061032864,213
7937843,5c4be542ff255bc80db4bef9b020f8e9,room,23613,emma donoghue,704696a561d43c562277a79adc85c8c0,4.0,"the reviews on the jacket said it was a book to read in one sitting - i didn't manage that, but i did read it in two sittings - one night for ""inside"" and one night for ""outside"". as i was reading the story inside the room, i found myself wondering how on earth the author was going to sustain a whole novel in such a small space - but in fact the escape, which i'd known had to happen at some point, occurred about halfway through. once it did happen, i found the second half of the book less satisfying, as jack and ma struggle to adjust to the outside world again. i don't spend much time with five-year-olds so i have no idea whether jack's speech and thoughts are true to that age or not, all i can say is that i was totall

## Perform Text Cleaning and Normalization

In [0]:
%pip install emoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/608.4 kB ? eta -:--:--
   ╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/608.4 kB ? eta -:--:--
   ━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/608.4 kB 871.6 kB/s eta 0:00:01
   ━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/608.4 kB 871.6 kB/s eta 0:00:01
   ━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.7/608.4 kB 511.6 kB/s eta 0:00:02
   ━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/608.4 kB 620.7 kB/s eta 0:00:01
   ━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/608.4 kB 495.1 kB/s eta 0:00:02
   ━━━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/608.4 kB 767.7 kB/s eta 0:00:01
   ━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━ 235.5/608.4 kB 923.9 kB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━ 307.2/608.4 kB 990.5 kB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━ 450.6/608.4 kB 1.3 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━━━━━ 512.0/608.4 kB 1.5 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 604.2

In [0]:
# Perform Text Cleaning and Normalization

from pyspark.sql.functions import udf, col, length, trim
from pyspark.sql.types import StringType
import re
import emoji

# Define a text-cleaning function
def clean_text(text):
    if text is None:
        return None
    
    # Convert to lowercase
    text = text.lower()
    
    # Remove URLs
    text = re.sub(r"http\S+|www\S+|https\S+", " [url] ", text)
    
    # Remove numbers
    text = re.sub(r"\d+", " [number] ", text)
    
    # Replace emojis with placeholder text
    text = emoji.replace_emoji(text, replace="[emoji]")
    
    # Remove punctuation and extra spaces
    text = re.sub(r"[^\w\s]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    
    return text

# Register the cleaning function as a UDF
clean_text_udf = udf(clean_text, StringType())

# Apply cleaning to the review_text column
df_cleaned_text = df.withColumn("cleaned_review_text", clean_text_udf(col("review_text")))

# Remove empty or very short reviews (<10 characters)
df_cleaned_text = df_cleaned_text.withColumn("cleaned_review_text", trim(col("cleaned_review_text"))) \
    .filter(length(col("cleaned_review_text")) >= 10)

# Show a preview
display(df_cleaned_text.select("review_text", "cleaned_review_text").limit(10))

review_text,cleaned_review_text
"shrk@ lnsh wljlwkwz; 'w kyf lzn@ skhyf@ 'n tsl bk l~ lhkm@? shrk@ lnsh wljlwkwz hw lktb lwhd wl`shrwn fy mjmw` m ktb `mr thr, wmn wjh@ nzry hw 'hl~ m ktb. lfkr@ n `mr thr khtr lmr@ dy nh yktb ns mftwh, hj@ kd@ `ml@ zy q`d@ m` whd shbk bttklmw fyh `n kl hj@, hkyt qdym@ shkhsy@ 'w trykhy@, skhry@ mn b`d mshhd l'flm, sys@, wklm `n lhb .. wlkwr@. `mr thr bythwl blktb d@ lsdyq l'y shkhs byqr' lktb, sh`wrk twl m nt msk lktb bn `mr thr mwjwd m`k wbyhky m`k fy 'mwr jyz thmk wjyz l, bs msh htkhsr hj@ bdan lw sm`th. 'kthr shy kn mthyr lldhsh@ blnsb@ ly hw qdr@ `mr thr `l~ stkhdm klmt mthl ""'tdhkr"" w ""blmnsb@"" lthtym syq lklm wkhlq syq jdyd bhky@ jdyd@ mkhtlf@ tmman `n m qblh, lrbt m byn lhkwy bsyt, fl `lq@ byn lklm `n lmlwkhy@ wbyn nsyh@ ljdw@ fy 'n ybhth `n lhb 'w ht~ byn lmlwkhy@ w`lqth blfzlk@ fy lhy@ bshkl `m. syqt `dyd@ wmkhtlf@ tmman `n b`d mtjm`yn fy mkn whd, zy m tjm` qt` sgyr@ mkhtlf@ lshkl wlhjm mn lbzl wfy lnhy@ yzhr lk lshkl lnhy'y mtkml wmtmsk, hw d@ blzbt lly `mlh `mr thr fy lktb d@. lktb bd' bshkw@ lktb mn zn@ skhyf@ tsytr `l~ 'dhnh, wnth~ bwswlh llhkm@ mn lhy@ wnth lznw@ lskhyf@, kyf lznwan@ skhyf@ 'n tsl blktb llhkm@ mn lhy@ bhdh lshkl? hdh hw m ydwr `nh kl shy fy lktb, kyf ymkn lshy shdyd lbst@ ymkn 'n ysbbh shy akhr 'shd bst@ mnh 'n yj`lk tsl l~ lhkm@, kyf tstty` rbt khywt l'hdth lsgyr@ ltshkyl hdth kbyr wwdh yj`lk tsl fy lnhy@ l~ lhkm@ mnh, tmman km wsl `mr thr bbst@ l~ lhkm@ mn mjrd znw@ sgyr@. 'slwb `mr thr shl `l~ lqry', whdh 'mr lys bhywn, nt mtkhyl whd yqr' mthlana `shryn ktb `lshn yfdl y`sr dmgh wyqdr ystkhrj mnh m ymkn 'n yj`lk tfkr fy 'mr ymkn tkwn mhmlh 'w d`t m byn 'mwr ktyr fy hytk tkwn shyfh 'kthr 'hmy@, mtkhyl kmn nh byhwl ywslw lk lhkm@ dy b'qs~ mjhwd mbdhwl mnh `lshn t`rf nt tlqth wtstw`bh b'ql mjhwd mnk? ldhlk ysthq `mr thr kl lshkr wltqdyr `l~ m ybdhlh mn mjhwd lktb@ mthl hdhh lktb shdyd@ l`dhwb@ wljml. wrbn ykhly lh rqy@ wthn lly blmnsb@ 'hd~ lh hdh lktb. b`d lqtbst lt~ tsthq ldhikr mn lktb: ""'sw m fy lhb, 'nh l y'ty 'bdan fy lwqt lmnsb, w'nh l ynthy, w'n wj`h hqyqy l ykmn fy lfrq wlkn fy ldhkryt, w'nh yj`lk tktshf 'nk gyrt 'shy knt mhm@ fy shkhsytk wtkhlyt `n 'shkhs knw mqrbyn wgyrt kht@ hytk l'sly@ mn 'jl shkhs lm y`d mwjwdan, w'nh yj`lk tftrd 'n hdhh l`lq@ (tjh whd) fl thsb hsban lnhyth l'y sbb ftqf fj'@ mdhhwlan b`d 'n knt tstql syr@ shkhs qrr fj'@ 'n ytkhls mnk fy mkn mqtw` lystkml ltryq bmfrdh .. 'm dh knt shb lsyr@ fsyj`lk lhb tktshf qym@ mn thbh mt'khran 'w km ql rmy sbry ""wbtdyt '`rf glwth lm rh"" ""yqwl lhkym kykws bn skndr fy ktbh ""ktb lnsyh@"" 'w ""qbws nmh"" blfrsy@: "" kn llh qdran `l~ 'n y`ty ldw bgyr lshms, w lmtr bgyr lshb, lkn kn l'mr klh `l~ 'ss lhkm@, w ldhlk lm ykhlq shyy'an bgyr lwst@, l'nh dh n`dmt lwst@ lm ykn hnk trtyb, w m lhkm@ l lqdr@ `l~ ltrtyb""."" ""sr l`mq 'mran mbtdhlan mn frt d`y'h twl lwqt, lshkhs ldhy ystn` l`mq hw 'kthr ky'n ykhsh~ 'n yzhr lljmy` sthyth, 'm mn ybhth `n lbst@ fhw shkhs ykfh llhrwb mn zlmt `myq@ fy rwhh ykhsh~ 'n ywrt fyh 'hdan, lbst@ hy 'kthr l'shy `mqan fy l`lm, lbst@ mjrd (shbryn myh), lkn(tt`b fyhm sfyn, w ttwh ltyrt) `l~ r'y 'gny@ lthmnynyt."" ""kyf tsbh khld hmdy? 4. shtri bkhms@ jny@ t`my@ w`nd khrwjk mn lmhl qm blqy'h fy lts@ 'thn thmyr lbtts wql llby'`: (m dmt qd khrbt hytk fy hdh lrkn mn l`lm fhytk khrb 'ynm hllt)"" ""lmtn` `n lhrk@ fy ntzr 'n ysqt rzqk f~ hjrk hw sw 'db m` llh, f'nt hn tkhtbr tqsymh llrzq, l'db 'n tthrk w'nt twqn tmmaa'n rzqk lys mkf'@ `l~ lhrk@ lknh ltms l'sbb dwn 'n tq` fy khtyy'@ `bd@ lsbb wnsyn lmsbb"" ""fth lrdr wl yqlqk gyb lshr@, shrt llh l tnqt` why n`m kbyr@, whw yqwl: l bd ln`mty mn 'khdh, wlw fkrt qlylan lfhmt lshr@ l`zym@ lmlhm@ fy 'nh khlq ljn@ 'wlan thm khlqk""",shrk lnsh wljlwkwz w kyf lzn skhyf n tsl bk l lhkm shrk lnsh wljlwkwz hw lktb lwhd wl shrwn fy mjmw m ktb mr thr wmn wjh nzry hw hl m ktb lfkr n mr thr khtr lmr dy nh yktb ns mftwh hj kd ml zy q d m whd shbk bttklmw fyh n kl hj hkyt qdym shkhsy w trykhy skhry mn b d mshhd l flm sys wklm n lhb wlkwr mr thr byth

## Extract Text-Based Features

In [0]:
gold_train_path = "abfss://lakehouse@goodreadsreviewsgen2.dfs.core.windows.net/gold/feature_v2/train"
df = spark.read.format("delta").load(gold_train_path)

In [0]:
%pip install nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/40.5 kB ? eta -:--:--
     ━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━ 20.5/40.5 kB 852.0 kB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━ 30.7/40.5 kB 726.9 kB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 319.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/57.7 kB ? eta -:--:--
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━━━ 51.2/57.7 kB 1.7 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/1.5 MB ? eta -:--:--
   ━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.1/1.5 MB 2.4 MB/s eta 0:00:01
   ━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.1/1.5 MB 2.3 MB/s eta 0:00:01
   ━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.1/1.5 MB 1.3 MB/s eta 0:00:02
   ━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.3/1.5 MB 1.9 MB/s eta 0:00:01
   ━━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.3/1.5 MB 2.2 MB/s eta 0:00:01
   ━

In [0]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/spark-f870426f-ae4f-40e6-b431-fa/nltk_data...


True

In [0]:
# Apply sentiment analysis using VADER
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from pyspark.sql.functions import udf, col
from pyspark.sql.types import FloatType

sid = SentimentIntensityAnalyzer()

# Define helper function
def get_sentiment_score(text, key):
    if text is None:
        return None
    scores = sid.polarity_scores(text)
    return float(scores[key])

# Register UDFs
sentiment_pos_udf = udf(lambda x: get_sentiment_score(x, "pos"), FloatType())
sentiment_neg_udf = udf(lambda x: get_sentiment_score(x, "neg"), FloatType())
sentiment_neu_udf = udf(lambda x: get_sentiment_score(x, "neu"), FloatType())
sentiment_compound_udf = udf(lambda x: get_sentiment_score(x, "compound"), FloatType())

# ✅ Apply to dataframe (fixed version)
df_features = df_cleaned_text \
    .withColumn("sentiment_pos", sentiment_pos_udf(col("cleaned_review_text"))) \
    .withColumn("sentiment_neg", sentiment_neg_udf(col("cleaned_review_text"))) \
    .withColumn("sentiment_neu", sentiment_neu_udf(col("cleaned_review_text"))) \
    .withColumn("sentiment_compound", sentiment_compound_udf(col("cleaned_review_text")))

# Preview sentiment results
display(df_features.select("cleaned_review_text", "sentiment_pos", "sentiment_neg", "sentiment_neu", "sentiment_compound").limit(10))

cleaned_review_text,sentiment_pos,sentiment_neg,sentiment_neu,sentiment_compound
shrk lnsh wljlwkwz w kyf lzn skhyf n tsl bk l lhkm shrk lnsh wljlwkwz hw lktb lwhd wl shrwn fy mjmw m ktb mr thr wmn wjh nzry hw hl m ktb lfkr n mr thr khtr lmr dy nh yktb ns mftwh hj kd ml zy q d m whd shbk bttklmw fyh n kl hj hkyt qdym shkhsy w trykhy skhry mn b d mshhd l flm sys wklm n lhb wlkwr mr thr bythwl blktb d lsdyq l y shkhs byqr lktb sh wrk twl m nt msk lktb bn mr thr mwjwd m k wbyhky m k fy mwr jyz thmk wjyz l bs msh htkhsr hj bdan lw sm th kthr shy kn mthyr lldhsh blnsb ly hw qdr mr thr l stkhdm klmt mthl tdhkr w blmnsb lthtym syq lklm wkhlq syq jdyd bhky jdyd mkhtlf tmman n m qblh lrbt m byn lhkwy bsyt fl lq byn lklm n lmlwkhy wbyn nsyh ljdw fy n ybhth n lhb w ht byn lmlwkhy w lqth blfzlk fy lhy bshkl m syqt dyd wmkhtlf tmman n b d mtjm yn fy mkn whd zy m tjm qt sgyr mkhtlf lshkl wlhjm mn lbzl wfy lnhy yzhr lk lshkl lnhy y mtkml wmtmsk hw d blzbt lly mlh mr thr fy lktb d lktb bd bshkw lktb mn zn skhyf tsytr l dhnh wnth bwswlh llhkm mn lhy wnth lznw lskhyf kyf lznwan skhyf n tsl blktb llhkm mn lhy bhdh lshkl hdh hw m ydwr nh kl shy fy lktb kyf ymkn lshy shdyd lbst ymkn n ysbbh shy akhr shd bst mnh n yj lk tsl l lhkm kyf tstty rbt khywt l hdth lsgyr ltshkyl hdth kbyr wwdh yj lk tsl fy lnhy l lhkm mnh tmman km wsl mr thr bbst l lhkm mn mjrd znw sgyr slwb mr thr shl l lqry whdh mr lys bhywn nt mtkhyl whd yqr mthlana shryn ktb lshn yfdl y sr dmgh wyqdr ystkhrj mnh m ymkn n yj lk tfkr fy mr ymkn tkwn mhmlh w d t m byn mwr ktyr fy hytk tkwn shyfh kthr hmy mtkhyl kmn nh byhwl ywslw lk lhkm dy b qs mjhwd mbdhwl mnh lshn t rf nt tlqth wtstw bh b ql mjhwd mnk ldhlk ysthq mr thr kl lshkr wltqdyr l m ybdhlh mn mjhwd lktb mthl hdhh lktb shdyd l dhwb wljml wrbn ykhly lh rqy wthn lly blmnsb hd lh hdh lktb b d lqtbst lt tsthq ldhikr mn lktb sw m fy lhb nh l y ty bdan fy lwqt lmnsb w nh l ynthy w n wj h hqyqy l ykmn fy lfrq wlkn fy ldhkryt w nh yj lk tktshf nk gyrt shy knt mhm fy shkhsytk wtkhlyt n shkhs knw mqrbyn wgyrt kht hytk l sly mn jl shkhs lm y d mwjwdan w nh yj lk tftrd n hdhh l lq tjh whd fl thsb hsban lnhyth l y sbb ftqf fj mdhhwlan b d n knt tstql syr shkhs qrr fj n ytkhls mnk fy mkn mqtw lystkml ltryq bmfrdh m dh knt shb lsyr fsyj lk lhb tktshf qym mn thbh mt khran w km ql rmy sbry wbtdyt rf glwth lm rh yqwl lhkym kykws bn skndr fy ktbh ktb lnsyh w qbws nmh blfrsy kn llh qdran l n y ty ldw bgyr lshms w lmtr bgyr lshb lkn kn l mr klh l ss lhkm w ldhlk lm ykhlq shyy an bgyr lwst l nh dh n dmt lwst lm ykn hnk trtyb w m lhkm l lqdr l ltrtyb sr l mq mran mbtdhlan mn frt d y h twl lwqt lshkhs ldhy ystn l mq hw kthr ky n ykhsh n yzhr lljmy sthyth m mn ybhth n lbst fhw shkhs ykfh llhrwb mn zlmt myq fy rwhh ykhsh n ywrt fyh hdan lbst hy kthr l shy mqan fy l lm lbst mjrd shbryn myh lkn tt b fyhm sfyn w ttwh ltyrt l r y gny lthmnynyt kyf tsbh khld hmdy number shtri bkhms jny t my w nd khrwjk mn lmhl qm blqy h fy lts thn thmyr lbtts wql llby m dmt qd khrbt hytk fy hdh lrkn mn l lm fhytk khrb ynm hllt lmtn n lhrk fy ntzr n ysqt rzqk f hjrk hw sw db m llh f nt hn tkhtbr tqsymh llrzq l db n tthrk w nt twqn tmmaa n rzqk lys mkf l lhrk lknh ltms l sbb dwn n tq fy khtyy bd lsbb wnsyn lmsbb fth lrdr wl yqlqk gyb lshr shrt llh l tnqt why n m kbyr whw yqwl l bd ln mty mn khdh wlw fkrt qlylan lfhmt lshr l zym lmlhm fy nh khlq ljn wlan thm khlqk,0.065,0.015,0.921,0.9875
i just went ahead and made myself a shelf for history mysteries with old houses and sad ladies because apparently it s my favourite fiction genre and like number of what i read,0.116,0.094,0.79,-0.0772
the devil s metal karina halle what first grabbed me about this book was the promise of a sexy rock star and a semi recluse ye determined young woman written for new adults if you have read s c stephens thoughtless you will understand the excitement a sexy angst ridden bad member can bring even from a literary perspective the beginning of the book was everything i had wanted even down to the over

In [0]:
from pyspark.sql.functions import col, lower, regexp_replace, split, size, explode, countDistinct, lit
from pyspark.sql import functions as F

# Step 1: Clean and tokenize
df_tokens = (
    df_features
    .withColumn("cleaned_text", lower(col("cleaned_review_text")))
    .withColumn("cleaned_text", regexp_replace(col("cleaned_text"), "[^a-zA-Z\\s]", ""))  # remove punctuation
    .withColumn("words", split(col("cleaned_text"), "\\s+"))
)

# Step 2: Compute term frequency (TF) manually
df_tf = df_tokens.withColumn("word", explode(col("words"))) \
                 .groupBy("book_id", "word") \
                 .agg(countDistinct("word").alias("tf"))

# Step 3: Compute document frequency (DF)
df_doc_freq = df_tokens.select(explode(col("words")).alias("word")) \
                       .groupBy("word") \
                       .agg(F.countDistinct(lit(1)).alias("df"))

# Step 4: Compute IDF and TF-IDF approximation
total_docs = df_tokens.count()
df_idf = df_doc_freq.withColumn("idf", F.log(lit(total_docs) / (col("df") + 1)))

# Join TF and IDF
df_tfidf = df_tf.join(df_idf, "word", "inner") \
                .withColumn("tfidf", col("tf") * col("idf"))

# Step 5: Aggregate TF-IDF scores back per review
df_tfidf_summary = df_tfidf.groupBy("book_id") \
                           .agg(F.mean("tfidf").alias("avg_tfidf_score"))

display(df_tfidf_summary.limit(5))
print("✅ TF-IDF features successfully computed with manual PySpark method (safe version).")

book_id,avg_tfidf_score
24747127,15.82408150651926
19099197,15.824081506519375
18373198,15.824081506519429
8813460,15.82408150651938
15743711,15.82408150651928


✅ TF-IDF features successfully computed with manual PySpark method (safe version).


In [0]:
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')

# Initialize VADER
sid = SentimentIntensityAnalyzer()

# Define UDFs
@udf(DoubleType())
def get_pos(text):
    return float(sid.polarity_scores(str(text))["pos"])

@udf(DoubleType())
def get_neg(text):
    return float(sid.polarity_scores(str(text))["neg"])

@udf(DoubleType())
def get_neu(text):
    return float(sid.polarity_scores(str(text))["neu"])

@udf(DoubleType())
def get_compound(text):
    return float(sid.polarity_scores(str(text))["compound"])

# Apply UDFs directly in Spark
df_features = (
    df_raw
    .withColumn("sentiment_pos", get_pos("review_text"))
    .withColumn("sentiment_neg", get_neg("review_text"))
    .withColumn("sentiment_neu", get_neu("review_text"))
    .withColumn("sentiment_compound", get_compound("review_text"))
    .withColumn("review_length_words", F.size(F.split(F.col("review_text"), " ")))
)

# Save to Silver layer
silver_features_path = "abfss://lakehouse@goodreadsreviewsgen2.dfs.core.windows.net/silver/features_v1/"
df_features.write.format("delta").mode("overwrite").save(silver_features_path)

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/spark-f870426f-ae4f-40e6-b431-fa/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [0]:
from pyspark.sql import functions as F

# Load the dataset that already has sentiment and review length
features_v1_path = "abfss://lakehouse@goodreadsreviewsgen2.dfs.core.windows.net/gold/features_v1/"
df_features = spark.read.format("delta").load(features_v1_path)

# Load your TF-IDF summary dataset (avg_tfidf_score per book)
tfidf_path = "abfss://lakehouse@goodreadsreviewsgen2.dfs.core.windows.net/gold/features_v2_tfidf/"
df_tfidf = spark.read.format("delta").load(tfidf_path)

# Merge both datasets on book_id
df_combined = df_features.join(df_tfidf, on="book_id", how="left")

# Check which columns exist now
print("✅ Columns in combined dataset:")
print(df_combined.columns[:30])

✅ Columns in combined dataset:
['book_id', 'review_id', 'title', 'author_id', 'author_name', 'user_id', 'rating', 'review_text', 'language_code', 'n_votes', 'date_updated', 'review_length', 'review_length_in_words', 'average_rating', 'number_of_reviews', 'avg_tfidf_score']


In [0]:
gold_path_tfidf = "abfss://lakehouse@goodreadsreviewsgen2.dfs.core.windows.net/gold/features_v2_tfidf/"
df_tfidf_summary.write.format("delta").mode("overwrite").save(gold_path_tfidf)

print("✅ TF-IDF feature summary successfully saved to Gold layer (features_v2_tfidf).")

✅ TF-IDF feature summary successfully saved to Gold layer (features_v2_tfidf).


In [0]:
# Redefine the Gold path variable
gold_path_tfidf = "abfss://lakehouse@goodreadsreviewsgen2.dfs.core.windows.net/gold/features_v2_tfidf/"

# Now read and preview the saved data
df_check = spark.read.format("delta").load(gold_path_tfidf)
display(df_check.limit(5))

book_id,avg_tfidf_score
22370139,15.824081506519375
20806501,15.824081506519395
7937843,15.824081506519718
4758093,15.824081506519464
9790160,15.824081506519384


In [0]:
# Load previous Gold features (Lab 3)
df_gold = spark.read.format("delta").load("abfss://lakehouse@goodreadsreviewsgen2.dfs.core.windows.net/gold/features_v1/")

# Load your new TF-IDF features
df_tfidf = spark.read.format("delta").load(gold_path_tfidf)

# Merge them on book_id
df_final = df_gold.join(df_tfidf, on="book_id", how="left")

display(df_final.limit(5))

book_id,review_id,title,author_id,author_name,user_id,rating,review_text,language_code,n_votes,date_updated,review_length,review_length_in_words,average_rating,number_of_reviews,avg_tfidf_score
7937843,5c4be542ff255bc80db4bef9b020f8e9,room,23613,emma donoghue,704696a561d43c562277a79adc85c8c0,4.0,"the reviews on the jacket said it was a book to read in one sitting - i didn't manage that, but i did read it in two sittings - one night for ""inside"" and one night for ""outside"". as i was reading the story inside the room, i found myself wondering how on earth the author was going to sustain a whole novel in such a small space - but in fact the escape, which i'd known had to happen at some point, occurred about halfway through. once it did happen, i found the second half of the book less satisfying, as jack and ma struggle to adjust to the outside world again. i don't spend much time with five-year-olds so i have no idea whether jack's speech and thoughts are true to that age or not, all i can say is that i was totally convinced.",eng,522469,2017-06-04,744,147,4.027589889640441,5618,15.824081506519718
22726728,8af9a5405d8f7de5f787a0f60610782b,"adjournment (fate, #1)",8261567,emersyn vallis,65d3b49e7e4d1248f487b081c2c535c4,4.0,"sydney chandler choose to leave home, to give up her family and most of all to leave the love of her life simon behind. he broke her heart, he made his choice now she had to make hers. she chose herself. she left it all and started her life anew. she never really left simon though he was always in her heart. the love she felt for him from their childhood never diminished. when fate throws them back into each other's sights can sydney forgive and forget? can simon prove to her he's worth the risk? my review - 4 stars this story is about sidney chandler and simon mcallister and the love they built when they were very young growing up together. somehow along the way tragedy had simon making a tough decision that tore at him for years. the plot takes us through the story of how fate gives two young lovers a second chance to get it right. ten years after simon broke sidney's heart fate brings then together and simon will stop at nothing to win sidney's trust and her heart. i loved how simon did everything he could to get sidney to notice him, to get her to see how he never forgot their connection. he knows her better than she knows herself. he has never truly let her out of his sight, watching her from a far keeping her safe and waiting for the day they could finally be together. i absolutely loved the character of simon and the fact that he was a triplet made him all the more appealing. three hot alpha males to love and adore. i very much enjoyed the banter between the three chandler sisters and the three mcallister brothers. lots of sexual tension, a little less heat then i am used to in the books i read but when the sex scenes come they are well written and steamy. something happened to simon and sidney that the author hints at it seems everyone else remembers the accident but sidney has blocked it out a bit. that part of the story confused me a little there were some holes in the story but there are four more books so i am sure the author will piece it all together. this one ends in a bit of a cliffhanger. all the books in the series are out so no waiting you can get the whole story all at once. received an arc in exchange for an honest review.",eng,180,2016-06-14,2191,418,3.8048780487804876,41,15.824081506519317
274400,f87ec9591ecacb0f7f49a31142d53260,"justice hall (mary russell, #6)",6760,laurie r. king,704696a561d43c562277a79adc85c8c0,4.0,"i'm so glad i gave laurie king a second chance after losing interest in the previous volume. justice hall was a great read. it's true i started skimming this book, too - but not until near the end of the last chapter, and only because i was getting sleepy and couldn't bear to put it down without finishing it! it's obvious the author has done her research about the early

In [0]:
# Check which columns are present in your current DataFrame
print("✅ Columns currently available in df_check:")
print(df_check.columns)

✅ Columns currently available in df_check:
['book_id', 'avg_tfidf_score']


In [0]:
from pyspark.sql import functions as F

# Load the Silver dataset (sentiment + length)
silver_path = "abfss://lakehouse@goodreadsreviewsgen2.dfs.core.windows.net/silver/features_v1/"
df_silver = spark.read.format("delta").load(silver_path)

# Load the TF-IDF Gold dataset
gold_path = "abfss://lakehouse@goodreadsreviewsgen2.dfs.core.windows.net/gold/features_v2_tfidf/"
df_tfidf = spark.read.format("delta").load(gold_path)

# Merge both on book_id (and review_id if both have it)
df_final = df_silver.join(df_tfidf, on="book_id", how="left")

# Preview the result
display(df_final.limit(5))
print("✅ Final combined dataset created successfully.")

book_id,date_added,date_updated,n_comments,n_votes,rating,read_at,review_id,review_text,started_at,user_id,sentiment_pos,sentiment_neg,sentiment_neu,sentiment_compound,review_length_words,avg_tfidf_score
6392944,Mon Jul 24 02:48:17 -0700 2017,Sun Jul 30 09:28:03 -0700 2017,0,6,3,Tue Jul 25 00:00:00 -0700 2017,5e212a62bced17b4dbe41150e5bb9037,"I haven't read a fun mystery book in a while and not sure I've ever read Poirot. Was looking for a fun read set in France while I was on holiday there and this didn't disappoint! Fast paced and good mystery. One that struck me was how similar Poirot is to Sherlock. They are both detectives, have a ex-military sidekick who is telling the story, and solve mysteries using their superior wit. Poirot seems like a French Sherlock. I'm curious if he was inspired by Sherlock.",Mon Jul 24 00:00:00 -0700 2017,8842281e1d1347389f2ab93d60773d4d,0.219,0.096,0.685,0.9018,87,15.824081506519331
18245960,Sun Jul 30 07:44:10 -0700 2017,Wed Aug 30 00:00:26 -0700 2017,1,28,5,Sat Aug 26 12:05:52 -0700 2017,dfdbb7b0eb5a7e4c26d59a937e2e5feb,"This is a special book. It started slow for about the first third, then in the middle third it started to get interesting, then the last third blew my mind. This is what I love about good science fiction - it pushes your thinking about where things can go. It is a 2015 Hugo winner, and translated from its original Chinese, which made it interesting in just a different way from most things I've read. For instance the intermixing of Chinese revolutionary history - how they kept accusing people of being ""reactionaries"", etc. It is a book about science, and aliens. The science described in the book is impressive - its a book grounded in physics and pretty accurate as far as I could tell. Though when it got to folding protons into 8 dimensions I think he was just making stuff up - interesting to think about though. But what would happen if our SETI stations received a message - if we found someone was out there - and the person monitoring and answering the signal on our side was disillusioned? That part of the book was a bit dark - I would like to think human reaction to discovering alien civilization that is hostile would be more like Enders Game where we would band together. I did like how the book unveiled the Trisolaran culture through the game. It was a smart way to build empathy with them and also understand what they've gone through across so many centuries. And who know a 3 body problem was an unsolvable math problem? But I still don't get who made the game - maybe that will come in the next book. I loved this quote: ""In the long history of scientific progress, how many protons have been smashed apart in accelerators by physicists? How many neutrons and electrons? Probably no fewer than a hundred million. Every collision was probably the end of the civilizations and intelligences in a microcosmos. In fact, even in nature, the destruction of universes must be happening at every second--for example, through the decay of neutrons. Also, a high-energy cosmic ray entering the atmosphere may destroy thousands of such miniature universes....""",Tue Aug 15 13:23:18 -0700 2017,8842281e1d1347389f2ab93d60773d4d,0.12,0.09,0.79,0.782,371,15.82408150651943
24375664,Fri Aug 25 13:55:02 -0700 2017,Mon Oct 09 08:55:59 -0700 2017,0,16,5,Sat Oct 07 00:00:00 -0700 2017,5cd416f3efc3f944fce4ce2db2290d5e,"Mind blowingly cool. Best science fiction I've read in some time. I just loved all the descriptions of the society of the future - how they lived in trees, the notion of owning property or even getting married was gone. How every surface was a screen. The undulations of how society responds to the Trisolaran threat seem surprising to me. Maybe its more the Chinese perspective, but I wouldn't have thought the ETO would exist in book 1, and I wouldn't have thought people would get so over-confident in our primitive fleet's chances given you have to think that with superior science they would 

✅ Final combined dataset created successfully.


In [0]:
final_gold_path = "abfss://lakehouse@goodreadsreviewsgen2.dfs.core.windows.net/gold/features_v2/"
df_final.write.format("delta").mode("overwrite").save(final_gold_path)
print("✅ Final combined feature dataset saved to Gold layer (features_v2).")

✅ Final combined feature dataset saved to Gold layer (features_v2).
